### Вспомогательные конструкции для добавления признаков

In [1]:
genre_groups = {
    'mainstream': [
        'pop', 'k-pop', 'dance', 'hip-hop', 'r-n-b',
        'edm', 'pop-film', 'indie-pop'
    ],
    'rock_metal': [
        'rock', 'metal', 'hard-rock', 'punk',
        'alternative', 'alt-rock', 'grunge',
        'heavy-metal', 'psych-rock', 'emo'
    ],
    'electronic': [
        'electronic', 'techno', 'house', 'trance',
        'dubstep', 'drum-and-bass', 'deep-house',
        'techno', 'electro', 'hardstyle'
    ],
    'chill': [
        'chill', 'ambient', 'acoustic', 'jazz',
        'blues', 'folk', 'singer-songwriter',
        'classical', 'piano', 'study', 'sleep'
    ],
    'world': [
        'latin', 'reggae', 'salsa', 'samba',
        'world-music', 'afrobeat', 'funk', 'disco',
        'country', 'bluegrass', 'tango'
    ],
    'extreme': [
        'metalcore', 'death-metal', 'black-metal',
        'hardcore', 'grindcore', 'industrial'
    ],
    'soundtrack': [
        'anime', 'disney', 'soundtrack', 'game',
        'comedy', 'children', 'kids', 'show-tunes',
        'j-pop', 'j-rock', 'k-pop', 'cantopop'
    ]
}

def map_genre_group(genre):
    for group, genres in genre_groups.items():
        if genre in genres:
            return group
    return 'other'

### Загрузка датасета, добавление признаков из feature engineering, разбиение на train/test

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector

df = pd.read_csv('dataset.csv')
df.dropna(inplace=True)
df.rename(columns={df.columns[0]: 'id'}, inplace=True) # первый столбец - id
pd.set_option('display.max_columns', None) # отображение всех колонок
target = 'popularity'

df['music_type'] = df['track_genre'].apply(map_genre_group)
df['duration_min'] = df['duration_ms'] / 1000 / 60
df['artist_count'] = df['artists'].str.split(';').str.len()
df['acousticness_pow_2.5'] = df['acousticness'] ** 2.5
df['chill_quiet'] = df['acousticness'] ** 2 * (1 - df['energy']) * (1 - df['speechiness'])
df['energy_pow_3.5'] = df['energy'] ** 3.5

num = ['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_min', 'artist_count',
       'acousticness_pow_2.5', 'chill_quiet', 'energy_pow_3.5'
]
cat = ['explicit', 'key', 'mode', 'time_signature', 'music_type']

X = df[num + cat]
y = df['popularity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Размерность: {X.shape}")

Размерность: (113999, 19)


### Функция для перебора параметров модели (валидация + тест)

In [3]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

def find_best_model(model, params, n_iter=25):
    num_indices = [X_train.columns.get_loc(col) for col in num]
    cat_indices = [X_train.columns.get_loc(col) for col in cat]
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_indices),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_indices)
        ]
    )
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    params_with_prefix = {f'model__{key}': value for key, value in params.items()}
    
    search = RandomizedSearchCV(pipeline, params_with_prefix, cv=kfold, scoring='r2', n_jobs=-1, refit=True, random_state=42)
    
    search.fit(X_train, y_train)
    
    best_model = search.best_estimator_

    base_score = 0.0560 # из feature engineering, после добавления признаков
    test_score = best_model.score(X_test, y_test)
    
    print(f"Лучшие параметры: {search.best_params_}")
    print(f"CV R^2 (средний): {search.best_score_:.4f}")
    print(f"Test R^2: {test_score:.4f}")
    print(f"Улучшение test R^2 на {(test_score - base_score):.4f}")
    
    return best_model, search.best_score_, test_score


### Вспомогательная конструкция для перебора моделей и параметров

In [4]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.neural_network import MLPRegressor

models = {
    # линейные
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet(),
    
    # деревья
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest Regressor': RandomForestRegressor(random_state=42),
    'Extra Trees Regressor': ExtraTreesRegressor(random_state=42),
    
    # градиентный бустинг
    'HistGradientBoosting Regressor': HistGradientBoostingRegressor(random_state=42),
    'XG Boost Regressor': XGBRegressor(random_state=42),
    'Light GBM Regressor': LGBMRegressor(random_state=42),
    'CatBoost Regressor': CatBoostRegressor(random_state=42, verbose=0),

    # нейросети
    'MLP Regressor': MLPRegressor(random_state=42, max_iter=500)
}

all_params = [
    # Linear Regression
    {},
    
    # Ridge
    {'alpha': [0.001, 0.005, 0.01, 0.05] + list(np.arange(0.1, 2.1, 0.1))},
    
    # Lasso
    {'alpha': [0.001, 0.005, 0.01, 0.05] + list(np.arange(0.1, 2.1, 0.1))},
    
    # ElasticNet
    {
        'alpha': [0.001, 0.005, 0.01, 0.05] + list(np.arange(0.1, 2.1, 0.1)),
        'l1_ratio': np.arange(0.1, 1.0, 0.1).tolist()
    },
    
    # Decision Tree
    {
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 10, 20, 30, 50],
        'min_samples_leaf': [1, 5, 20],
        'max_features': ['sqrt', 'log2', None]
    },
    
    # Random Forest Regressor
    {
        'n_estimators': [100, 200],
        'max_depth': [None, 15, 20],
        'min_samples_split': [2, 10, 50],
        'min_samples_leaf': [1, 5, 20],
        'max_features': ['sqrt', 0.5, 0.7],
        'bootstrap': [True],
        'n_jobs': [-1]
    },
    
    # Extra Trees Regressor
    {
        'n_estimators': [100, 200],
        'max_depth': [None, 15, 20],
        'min_samples_split': [2, 10],
        'min_samples_leaf': [1, 5],
        'max_features': ['sqrt', 0.5],
        'bootstrap': [True],
        'n_jobs': [-1]
    },
    
    # HistGradientBoosting Regressor
    {
        'learning_rate': [0.05, 0.1, 0.2],
        'max_iter': [100, 200],
        'max_depth': [None, 7, 9],
        'min_samples_leaf': [20, 50, 100],
        'max_leaf_nodes': [None, 63],
        'l2_regularization': [0.0, 0.1, 1.0]
    },
    
    # XG Boost Regressor
    {
        'n_estimators': [100, 200],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [5, 7, 9],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'gamma': [0, 0.5],
        'reg_alpha': [0, 0.5],
        'reg_lambda': [1, 2],
        'n_jobs': [-1],
        'tree_method': ['hist']
    },
    
    # Light GBM Regressor
    {
        'n_estimators': [100, 200],
        'learning_rate': [0.05, 0.1, 0.2],
        'num_leaves': [31, 63],
        'max_depth': [None, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'reg_alpha': [0, 0.5],
        'reg_lambda': [0, 0.5],
        'n_jobs': [-1]
    },
    
    # CatBoost Regressor
    {
        'iterations': [200, 500],
        'learning_rate': [0.05, 0.1, 0.2],
        'depth': [6, 8],
        'l2_leaf_reg': [3, 7],
        'border_count': [128],
        'verbose': [False],
    },
    
    # MLP Regressor - КРАЙНЕ УПРОЩЕНО
    {
        'hidden_layer_sizes': [(50,), (100,)],
        'activation': ['relu'],
        'alpha': [0.001, 0.01],
        'learning_rate_init': [0.001, 0.01],
        'batch_size': [256, 512],
        'solver': ['adam'],
        'early_stopping': [True],
        'max_iter': [200],
        'n_iter_no_change': [10]
    }
]

### Перебор и поиск лучших моделей из разных категорий

In [5]:
import warnings
warnings.filterwarnings('ignore')
for (model_name, model), params in zip(models.items(), all_params):
    print(model_name)
    find_best_model(model, params)
    print()

Linear Regression
Лучшие параметры: {}
CV R^2 (средний): 0.0587
Test R^2: 0.0560
Улучшение test R^2 на 0.0000

Ridge
Лучшие параметры: {'model__alpha': np.float64(1.8000000000000003)}
CV R^2 (средний): 0.0587
Test R^2: 0.0560
Улучшение test R^2 на 0.0000

Lasso
Лучшие параметры: {'model__alpha': 0.001}
CV R^2 (средний): 0.0587
Test R^2: 0.0560
Улучшение test R^2 на 0.0000

ElasticNet
Лучшие параметры: {'model__l1_ratio': 0.7000000000000001, 'model__alpha': 0.005}
CV R^2 (средний): 0.0586
Test R^2: 0.0559
Улучшение test R^2 на -0.0001

Decision Tree
Лучшие параметры: {'model__min_samples_split': 20, 'model__min_samples_leaf': 20, 'model__max_features': None, 'model__max_depth': None}
CV R^2 (средний): 0.1477
Test R^2: 0.1658
Улучшение test R^2 на 0.1098

Random Forest Regressor
Лучшие параметры: {'model__n_jobs': -1, 'model__n_estimators': 100, 'model__min_samples_split': 2, 'model__min_samples_leaf': 1, 'model__max_features': 0.7, 'model__max_depth': 20, 'model__bootstrap': True}
CV R^

# Выводы
**Лучшие модели**
1. HistGradientBoostingRegressor: Test R^2 = 0.4842 (+0.4282 от базовой модели)
2. XGBRegressor: Test R^2 = 0.4462 (+0.3902)
3. RandomForestRegressor: Test R^2 = 0.4137 (+0.3577)

Статистика по видам моделей:
- Линейные модели не работают
- Деревья и ансамбли работают хорошо
- Модели градиентного бустинга - лучшие
- Нейронные сети лучше линейных моделей, но хуже остальных

Изменение метрики R^2 на тесте:
- Feature engineering дал прирост с 0.0471 до 0.0560 (+0.0089)
- Подбор модели дал улучшение с 0.0560 до 0.4842 (в 8.6 раз!)

Таким образом, для предсказания популярности нужны сложные нелинейные модели. Градиентный бустинг и ансамбли деревьев дают наилучший результат.